# Kalman filter for state estimation and occlusion percentage
We get in input the output of the yolo, so a set of bounding box, defined with their center (x,y) coordinates
and the width and the height of this rectangle.
By evaluating the percentage of the area visible of each bounding box, we can say if a pedestrian/cyclist/car is occluded, and if it is so, we'll call the state estimation function to predict its movement under the occlusion.

## YOLOv5 Detection Results for Image: x512.png
Format: [class] [confidence_score] [x_center] [y_center] [width] [height]

Vehicles
vehicle 0.94 0.54 0.65 0.10 0.15;
vehicle 0.93 0.48 0.63 0.10 0.15;
vehicle 0.91 0.50 0.60 0.10 0.15;
vehicle 0.88 0.45 0.58 0.10 0.15;
vehicle 0.84 0.40 0.55 0.10 0.15;
vehicle 0.70 0.38 0.53 0.10 0.15;

Pedestrians
pedestrian 0.88 0.57 0.68 0.05 0.10;
pedestrian 0.80 0.59 0.70 0.05 0.10;
pedestrian 0.76 0.52 0.69 0.05 0.10;
pedestrian 0.75 0.60 0.64 0.05 0.10;
pedestrian 0.71 0.61 0.66 0.05 0.10;
pedestrian 0.67 0.62 0.62 0.05 0.10;
pedestrian 0.66 0.63 0.61 0.05 0.10;

# Occlusion rate
Performance: This approach has 
O(n2) complexity due to the nested loop for each pair of bounding boxes. For large numbers of objects,we may want to optimize it by using spatial data structures like a KD-tree.
Partial Occlusion: This method doesn’t account for partial occlusion from objects with irregular shapes. It assumes that occlusions are always rectangular.


In [3]:
import cv2
import numpy as np

# Function to calculate intersection area between a box and the overlay region
def calculate_occlusion_area(box, overlay_rect):
    x1 = max(box[0], overlay_rect[0])
    y1 = max(box[1], overlay_rect[1])
    x2 = min(box[2], overlay_rect[2])
    y2 = min(box[3], overlay_rect[3])
    
    intersection_width = max(0, x2 - x1)
    intersection_height = max(0, y2 - y1)
    return intersection_width * intersection_height

# Load the image
image = cv2.imread("urban-detection-master/res/s512.png")

# Path to YOLO output file
file_path = "yolo_output.txt" 

# Image dimensions for denormalization
img_height, img_width, _ = image.shape

# Parse bounding boxes from the file
bounding_boxes = []
with open(file_path, "r") as file:
    for line in file:
        if line.startswith("#") or line.strip() == "":
            continue
        
        parts = line.strip().split()
        label = parts[0]
        confidence = float(parts[1])
        x_center = float(parts[2])
        y_center = float(parts[3])
        width = float(parts[4])
        height = float(parts[5])
        
        # Denormalize coordinates
        x_center = int(x_center * img_width)
        y_center = int(y_center * img_height)
        box_width = int(width * img_width)
        box_height = int(height * img_height)
        
        # Calculate top-left and bottom-right corners of the bounding box
        x1 = int(x_center - box_width / 2)
        y1 = int(y_center - box_height / 2)
        x2 = int(x_center + box_width / 2)
        y2 = int(y_center + box_height / 2)
        
        # Append the bounding box with label and confidence
        bounding_boxes.append((label, x1, y1, x2, y2,x_center,y_center))

'''
# Calculate occlusion rates
occlusion_rates = []
for i, (label_i, x1_i, y1_i, x2_i, y2_i) in enumerate(bounding_boxes):
    box_i_area = (x2_i - x1_i) * (y2_i - y1_i)
    occluded_area = 0
    
    # Compare with all other boxes
    for j, (label_j, x1_j, y1_j, x2_j, y2_j) in enumerate(bounding_boxes):
        if i != j:  # Skip self-comparison
            occlusion_area = calculate_occlusion_area((x1_i, y1_i, x2_i, y2_i), (x1_j, y1_j, x2_j, y2_j))
            occluded_area += intersection_area
            # sometimes oclluded_area > 100% because same parts of the analyzed bounding box are
            # occluded by more than 1 bounding box.
    
    # Calculate occlusion rate as a percentage
    occlusion_rate = (occluded_area / box_i_area) * 100 if box_i_area > 0 else 0
    occlusion_rates.append((label_i, occlusion_rate))
    
    # Display occlusion rate
    print(f"Label: {label_i}, Occlusion Rate: {occlusion_rate:.2f}%")

# Display the final image
cv2.imshow("Occlusion Rates", image)
cv2.waitKey(0)
cv2.destroyAllWindows()
'''

'\n# Calculate occlusion rates\nocclusion_rates = []\nfor i, (label_i, x1_i, y1_i, x2_i, y2_i) in enumerate(bounding_boxes):\n    box_i_area = (x2_i - x1_i) * (y2_i - y1_i)\n    occluded_area = 0\n    \n    # Compare with all other boxes\n    for j, (label_j, x1_j, y1_j, x2_j, y2_j) in enumerate(bounding_boxes):\n        if i != j:  # Skip self-comparison\n            occlusion_area = calculate_occlusion_area((x1_i, y1_i, x2_i, y2_i), (x1_j, y1_j, x2_j, y2_j))\n            occluded_area += intersection_area\n            # sometimes oclluded_area > 100% because same parts of the analyzed bounding box are\n            # occluded by more than 1 bounding box.\n    \n    # Calculate occlusion rate as a percentage\n    occlusion_rate = (occluded_area / box_i_area) * 100 if box_i_area > 0 else 0\n    occlusion_rates.append((label_i, occlusion_rate))\n    \n    # Display occlusion rate\n    print(f"Label: {label_i}, Occlusion Rate: {occlusion_rate:.2f}%")\n\n# Display the final image\ncv2.imsh

In [4]:
# let's consider the overlap from an entire image (Wazowzky)
overlap_image = cv2.imread("Overlap_image.png")
# Define overlay image position and size on the original image
overlay_x, overlay_y = 200, 150  # Top-left corner of the overlay on the original image (assumed)
overlap_image = cv2.resize(overlap_image,(round(overlap_image.shape[1]/3), round(overlap_image.shape[0]/3)))
overlay_width, overlay_height = overlap_image.shape[1], overlap_image.shape[0]

# Define overlay region as a bounding box
overlay_rect = (overlay_x, overlay_y, overlay_x + overlay_width, overlay_y + overlay_height)

# Calculate occlusion rate for each bounding box
occlusion_rates = []
for label, x1, y1, x2, y2, x_center, y_center in bounding_boxes:
    box_area = (x2 - x1) * (y2 - y1)
    occlusion_area = calculate_occlusion_area((x1, y1, x2, y2), overlay_rect)
    
    occlusion_rate = (occlusion_area / box_area) * 100 if box_area > 0 else 0
    
    if occlusion_rate == 100:
        occlusion_rates.append((label, x1, y1, x2, y2, x_center, y_center, 0)) # not visible at all
    if occlusion_rate<100 and occlusion_rate >= 50:
        occlusion_rates.append((label, x1, y1, x2, y2, x_center, y_center, 1)) # partially visible
    if occlusion_rate<50 and occlusion_rate >= 0:
        occlusion_rates.append((label, x1, y1, x2, y2, x_center, y_center, 2)) # mostly visible
    else:
        occlusion_rates.append((label, x1, y1, x2, y2, x_center, y_center, 3)) # totally visible
    
    # Print results
    print(f"Label: {label}, Confidence: {confidence:.2f}, Occlusion Rate: {occlusion_rate:.2f}%")

'''# display the overlay region on the original image
cv2.rectangle(image, (overlay_x, overlay_y), (overlay_x + overlay_width, overlay_y + overlay_height), (0, 0, 255), 2)
for label, x1, y1, x2, y2 in bounding_boxes:
    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
    cv2.putText(image, f"{label} ", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)'''

print(occlusion_rates)


Label: vehicle, Confidence: 0.41, Occlusion Rate: 11.85%
Label: vehicle, Confidence: 0.41, Occlusion Rate: 51.30%
Label: vehicle, Confidence: 0.41, Occlusion Rate: 51.56%
Label: vehicle, Confidence: 0.41, Occlusion Rate: 93.59%
Label: vehicle, Confidence: 0.41, Occlusion Rate: 100.00%
Label: vehicle, Confidence: 0.41, Occlusion Rate: 100.00%
Label: pedestrian, Confidence: 0.41, Occlusion Rate: 0.00%
Label: pedestrian, Confidence: 0.41, Occlusion Rate: 0.00%
Label: pedestrian, Confidence: 0.41, Occlusion Rate: 2.49%
Label: pedestrian, Confidence: 0.41, Occlusion Rate: 0.00%
Label: pedestrian, Confidence: 0.41, Occlusion Rate: 0.00%
Label: pedestrian, Confidence: 0.41, Occlusion Rate: 0.00%
Label: pedestrian, Confidence: 0.41, Occlusion Rate: 0.00%
Label: construction, Confidence: 0.41, Occlusion Rate: 84.82%
[('vehicle', 366, 299, 440, 377, 403, 338, 2), ('vehicle', 322, 288, 396, 366, 359, 327, 1), ('vehicle', 322, 288, 396, 366, 359, 327, 3), ('vehicle', 337, 273, 411, 351, 374, 312, 

# Kalman filter


In [27]:

def filter_red_color(frame):
    # Convert the frame to HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Define the red color range
    lower_red1 = np.array([0, 100, 50])
    upper_red1 = np.array([7, 255, 255])
    lower_red2 = np.array([173, 100, 50])
    upper_red2 = np.array([180, 255, 255])

    # Create masks for both red ranges
    mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
    mask2 = cv2.inRange(hsv, lower_red2, upper_red2)

    # Combine the two masks
    mask = mask1 + mask2

    # Apply the mask to get only red parts of the image
    result = cv2.bitwise_and(frame, frame, mask=mask)

    return result

def initialize_kalman():
    kalman = {
        "x": np.array([0,
              0,
              0,
              0,
              0,
              0]),  # State vector
        "P": 1000 * np.eye(6),  # Initial uncertainty, a random high number
        "F": np.array([[1, 1, 0.5, 0, 0, 0],
                       [0, 1, 1, 0, 0, 0],
                       [0, 0, 1, 0, 0, 0],
                       [0, 0, 0, 1, 1, 0.5],
                       [0, 0, 0, 0, 1, 1],
                       [0, 0, 0, 0, 0, 1]]),  # Transition matrix
        "u": np.zeros(6),  # External motion
        "H": np.array([[1, 0, 0, 0, 0, 0],  # Observe x position
                       [0, 0, 0, 1, 0, 0]]),  # Observe y position
        "R": np.eye(2),  # Measurement uncertainty
        "I": np.eye(6)  # Identity matrix
    }
    return kalman

# Function to update the Kalman filter
def update(kalman, Z):
    x, P, H, R, I = kalman["x"], kalman["P"], kalman["H"], kalman["R"], kalman["I"]
    
    # Measurement residual y
    y = Z - np.dot(H, x)
    
    # Residual covariance S
    S = np.dot(H, np.dot(P, H.T)) + R
    
    # Kalman gain K
    K = np.dot(P, np.dot(H.T, np.linalg.inv(S)))
    
    # Update state estimate x
    x = x + np.dot(K, y)
    
    # Update uncertainty P
    P = np.dot(I - np.dot(K, H), P)
    
    kalman["x"], kalman["P"] = x, P
    return kalman

# Function to predict the next state
def predict(kalman):
    x, P, F, u = kalman["x"], kalman["P"], kalman["F"], kalman["u"]
    
    # Predict state x
    x = np.dot(F, x) + u
    
    # Predict uncertainty P
    P = np.dot(F, np.dot(P, F.T))
    
    kalman["x"], kalman["P"] = x, P
    return kalman

import cv2

def draw_multiple_bounding_boxes(image, boxes, color=(0, 255, 0), thickness=2):
    """
    Draws multiple bounding boxes on an image.

    Parameters:
        image (numpy array): The image on which the bounding boxes will be drawn.
        boxes (list): A list of dictionaries. Each dictionary should contain:
                      - 'center': A tuple (cx, cy) representing the center of the bounding box.
                      - 'width': The width of the bounding box.
                      - 'height': The height of the bounding box.
                      - 'label' (optional): A string to display above the bounding box.
        color (tuple): The color of the bounding boxes in BGR format (default is green).
        thickness (int): The thickness of the bounding box lines (default is 2).

    Returns:
        numpy array: The image with the bounding boxes drawn.
    """
    for box in boxes:
        # Extract parameters
        center = box['center']
        width = box['width']
        height = box['height']
        #label = box.get('label', None)  # Optional label
        
        # Calculate top-left and bottom-right coordinates
        cx, cy = center
        x1 = int(cx - width / 2)
        y1 = int(cy - height / 2)
        x2 = int(cx + width / 2)
        y2 = int(cy + height / 2)
        
        # Draw the rectangle
        cv2.rectangle(image, (x1, y1), (x2, y2), color, thickness)
        
        # Draw the label if provided
        if label:
            cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, thickness)
    
    return image

'''
# Load the video
cap = cv2.VideoCapture('rolling_ball.mp4')
if not cap.isOpened():
    print("Cannot open video")
    exit()

# Sharpening kernel
# to emphasize the central pixels
sharpen_kernel = np.array([[-1, -1, -1],
                           [-1,  9, -1],
                           [-1, -1, -1]])

# Initialize radius with a default value
radius = 0

# Looping through all the frames
while True:
    ret, frame = cap.read()
    if not ret:
        break
        
    ### Detect the ball ###
    # Filter by red color first
    red_frame = filter_red_color(frame)

    # Convert the red frame to grayscale for edge detection
    gray = cv2.cvtColor(red_frame, cv2.COLOR_BGR2GRAY)

    # Apply a Gaussian blur
    gray_blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Detect edges
    edges = cv2.Canny(gray_blurred, 30, 50)

    # Apply HoughCircles to detect circles
    pieces = cv2.HoughCircles(edges, cv2.HOUGH_GRADIENT, 1, 80, param1=100, param2=20, minRadius=40, maxRadius=125)

    # We now display the ball if it was found
    if pieces is not None:
        # Convert the (x, y, radius) values of the circles to integers
        pieces = np.uint16(np.around(pieces))
    
        # Draw the circles on the original image
        circle = pieces[0, 0]
        center = (circle[0], circle[1])
        radius = circle[2]

        cv2.circle(frame, center, radius, (0, 255, 0), 5)  # Green circle with thickness 2
    
        ### If the ball is found, update the Kalman filter ###
        Z = np.array(center)  # Z holds the "measured position" of the ball
        x, P = update(x, P, Z, H, R)
    
    ### Predict the next state
    x, P = predict(x, P, F, u)
    predicted_center = (int(x[0]), int(x[3]))
    ### Draw the predicted state on the image frame ###
    cv2.circle(frame, predicted_center, radius, (0, 0, 255), 5)
    

    # Get original video dimensions
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    # Set a desired maximum window width or height, while maintaining the aspect ratio
    desired_width = 1000  # Change this as needed
    
    # Calculate the appropriate height based on the aspect ratio
    aspect_ratio = frame_height / frame_width
    new_height = int(desired_width * aspect_ratio)
    
    # Set up the window for display and resize it, keeping the aspect ratio
    cv2.namedWindow('Frame', cv2.WINDOW_NORMAL)  # Create a window
    cv2.resizeWindow('Frame', desired_width, new_height)  # Resize keeping the aspect ratio

    # Show the frame
    cv2.imshow('Frame', frame)
    cv2.waitKey(150)
    
cap.release()
cv2.destroyAllWindows()'''

'\n# Load the video\ncap = cv2.VideoCapture(\'rolling_ball.mp4\')\nif not cap.isOpened():\n    print("Cannot open video")\n    exit()\n\n# Sharpening kernel\n# to emphasize the central pixels\nsharpen_kernel = np.array([[-1, -1, -1],\n                           [-1,  9, -1],\n                           [-1, -1, -1]])\n\n# Initialize radius with a default value\nradius = 0\n\n# Looping through all the frames\nwhile True:\n    ret, frame = cap.read()\n    if not ret:\n        break\n        \n    ### Detect the ball ###\n    # Filter by red color first\n    red_frame = filter_red_color(frame)\n\n    # Convert the red frame to grayscale for edge detection\n    gray = cv2.cvtColor(red_frame, cv2.COLOR_BGR2GRAY)\n\n    # Apply a Gaussian blur\n    gray_blurred = cv2.GaussianBlur(gray, (5, 5), 0)\n\n    # Detect edges\n    edges = cv2.Canny(gray_blurred, 30, 50)\n\n    # Apply HoughCircles to detect circles\n    pieces = cv2.HoughCircles(edges, cv2.HOUGH_GRADIENT, 1, 80, param1=100, param2=2

In [28]:
boxes=[]
for label,x1, y1, x2, y2, x_center, y_center,occlusion_rate in occlusion_rates:
    if occlusion_rate < 2: # so 0 or 1
        #kalman filter applied, we have to initialize the matrices (x,P,u,F,P...) for each bounding box
        # and the state (position, velocity, etc.) and uncertainty (P matrix) for each object need to be tracked independently.
        width = x2-x1
        height = y2-y1
        kalman = initialize_kalman()
        Z = [x_center,y_center]   # it has to be 1x2 vector
        kalman = update(kalman,Z)
        kalman = predict(kalman)
        new_x = kalman['x']
        new_center = (round(new_x[0]),round(new_x[3]))
        # now we have to expand the dictionary putting in the new coordinate of predicted the center
        boxes.append({new_center, width, height})
    else:
       continue